# Code to produce the cutouts for the MIRI data (DONE)

Let's start with the imports

In [ ]:
from astropy.io import fits
from astropy.coordinates import SkyCoord
from astropy.wcs import WCS
from astropy.nddata import Cutout2D
import astropy.units as u
import matplotlib.pyplot as plt
import numpy as np
import os
import glob

So takin this conversion of pixels to angular resolution we find that MIRI pixels are on average about 3.5 times larger than NIRCam pixels, so we just need to divide 167 by 3.55, which leaves us with ~48 pixels in MIRI.

# Define function to produce cutouts automatically

In [ ]:
def produce_cutouts(cat, indir, output_dir, survey, x_pixels, filter, nan_thresh=0.4):
    """Function that reads in all the .fits files in a folder and produces cutouts, 
        which are then stored in the output directory


    Args:
        cat (string): catalogue.fits file that contains galaxy positions and IDs
        indir (string): Input directory for the .fits files from which the cutouts are taken
        output_dir (string): Output directory in which the cutouts should be stored
        survey (string): Either 'primer' for PRIMER or 'cweb' for COSMOS-Web
        x_pixels (int): Side length of the cutouts in pixels (produces a square frame)
        filter (string): Specific filter used for the imaging
        nan_thresh (float, optional): Relative amount of nan entries allowed in the data of 
            each cutout. Defaults to 0.4.
    """
    
    # Load target catalogue
    with fits.open(cat) as catalog_hdul:
        catalog_hdul.info()
        cat_data = catalog_hdul[1].data  # Extract data from table
        ids = cat_data['id']
        ra = cat_data['ra']
        dec = cat_data['dec']  
    
    filter_l = filter.lower()
    
    if '.fits' in indir:
        fits_files = [indir]
    else:
        fits_files = glob.glob(os.path.join(indir, f"*{filter_l}*.fits"))
        print(f"Found {len(fits_files)} FITS files from the {survey} survey with filter {filter} in directory {indir}.")
        print("Processing:")
    for f in fits_files:
        print(f"{f}")
    
    # Create output directory
    os.makedirs(output_dir, exist_ok=True)
    print(f"Files will be saved to {output_dir}.")

    counts = 0
    total = len(ra)

    cutout_size = (x_pixels, x_pixels)
    
    if "rot" in indir: x = 0
    else: x = 1
    
    for data in fits_files:
        with fits.open(data) as miri_hdul:
            miri_data = miri_hdul[x].data
            miri_header = miri_hdul[x].header
            wcs = WCS(miri_header)

        # Loop over all galaxies first
        for i in range(len(ra)):
            target_coord = SkyCoord(ra[i], dec[i], unit=(u.deg, u.deg))
            found = False  # Track if the galaxy was mapped

            # Check if the galaxy is inside the mapped region
            x, y = wcs.world_to_pixel(target_coord)
            if (0 <= x < miri_data.shape[1]) and (0 <= y < miri_data.shape[0]):
                found = True  # The galaxy was mapped

                # Extract the cutout
                cutout = Cutout2D(miri_data, target_coord, cutout_size, wcs=wcs, mode="partial")

                # Ensure cutout is valid
                if cutout.data is None or cutout.data.size == 0:
                    #print(f"Skipping {ids[i]}: Empty cutout")
                    continue
                
                # Check for NaN ratio
                nans = np.isnan(cutout.data).sum()
                nan_ratio = nans / cutout.data.size
                
                if nan_ratio < nan_thresh:
                    # Save PNG preview
                    plt.figure(figsize=(6, 6))
                    plt.imshow(cutout.data, origin="lower", cmap="gray")
                    plt.colorbar()
                    plt.title(f'{survey}: {ids[i]} at {filter}')

                    png_filename = os.path.join(output_dir, f"{ids[i]}_{filter}_cutout_{survey}.png")
                    plt.savefig(png_filename)
                    plt.close()

                    counts += 1

                    # Adjust header CRPIX values for the cutout
                    # Get the new reference pixel
                    new_crpix = cutout.wcs.world_to_pixel(target_coord)
                    new_header = cutout.wcs.to_header()
                    new_header['CRPIX1'] = int(np.round(new_crpix[0]))
                    new_header['CRPIX2'] = int(np.round(new_crpix[1]))
                    
                    # Save the cutout FITS file
                    hdu_list = fits.HDUList()
                    hdu_list.append(fits.PrimaryHDU(header=miri_hdul[0].header))
                    hdu_list.append(fits.ImageHDU(data=cutout.data, header=cutout.wcs.to_header()))

                    for ext in range(2, len(miri_hdul)):
                        if miri_hdul[ext].data is not None:
                            hdu_list.append(fits.ImageHDU(data=miri_hdul[ext].data, header=miri_hdul[ext].header))

                    fits_filename = os.path.join(output_dir, f"{ids[i]}_{filter}_cutout_{survey}.fits")
                    hdu_list.writeto(fits_filename, overwrite=True)

                    #print(f"Saved: {ids[i]} at {filter}")
                else:
                    #print(f"Skipping {ids[i]}: Too many NaNs ({nan_ratio:.2f})")
                    continue
            else:
                continue  # Galaxy is outside the mapped region

        if not found:
            #print(f"Skipping {ids[i]}: Not found in any FITS file")
            continue
        
    print(f"Produced cutouts for {counts} of {total} galaxies in the catalogue.")

        

# Create cutouts!!

Note that the number of MIRI pixels needs to be reduced from 48 to 45 when dealing with the expanded and rotated .fits file as a source of the cutouts. This is due to the resampling of the pixels.

In [ ]:
primer_dir = '/home/bpc/University/master/Red_Cardinal/MIRI/PRIMER/'
cweb_dir = "/home/bpc/University/master/Red_Cardinal/MIRI/COSMOS-Web/"
catalogue = './../cat_targets.fits'
output_dir = './../cutouts_rotated/'

exp_rot = "./../exp_rot/jw01837-o004_t004_miri_f770w_i2d_exp_rot.fits"

#produce_cutouts(catalogue, primer_dir, output_dir, 'primer', 48, 'F770W')
#produce_cutouts(catalogue, primer_dir, output_dir, 'primer', 48, 'F1800W')
#produce_cutouts(catalogue, cweb_dir, output_dir, 'cweb', 48, 'F770W')
#produce_cutouts(catalogue, primer_dir, test_dir, 'primer', 48, 'F770W')


produce_cutouts(catalogue, exp_rot, output_dir, 'primer', 45, 'F770W')